In [1]:
import pandas as pd
import numpy as np
from numba import jit, cuda
import tqdm
import ijson
import warnings
warnings.filterwarnings("ignore")

def count_neg_emojis(text):
    words = set(word for word in text.split(' '))
    nb_negative = len(words.intersection(neg_emojis))
    return nb_negative

def count_neg_emojis(text):
    words = set(word for word in text.split(' '))
    nb_negative = len(words.intersection(neg_emojis))
    return nb_negative

{'👹', '❗', '💥', '☠', '🔫', '📛', '😠', '😤', '💢', '🚫', '☣', '🔞', '☢', '🔪', '🧨', '👺', '💀', '🤡', '🤢', '💩', '😣', '🪓', '‼', '🗡', '👎', '🤮', '😈', '👿', '⛔', '😡', '🤬', '🖕', '⁉'}


We define the set of negative emojis as follow:

In [ ]:
neg_emojis = set(["\U0001F52B","\U0001F92E","\U0001F922","\U0001F623","\U0001F624","\U0001F621","\U0001F620","\U0001F92C","\U0001F608","\U0001F47F","\U0001F480","\U00002620","\U0001F4A9","\U0001F921","\U0001F479","\U0001F47A","\U0001F4A5","\U0001F4A2","\U0001F595","\U0001F44E","\U0001F52A","\U0001F9E8","\U0001FA93","\U0001F5E1","\U0001F51E","\U000026D4","\U0001F6AB","\U00002623","\U00002622","\U00002757","\U0000203C","\U00002049","\U0001F4DB"])
print(neg_emojis)

In [25]:
def plot_count_neg_emojis(chunks):
    for chunk in chunks:
        df_nb_neg_emo = chunk.apply(lambda x : count_negative_emojis(x['description'],x['tags'],x['title']), axis=1)
        df_nb_neg_emo_non_null = df_nb_neg_emo[df_nb_neg_emo > 0]
        df_nb_neg_emo_non_null.plot.hist() 
        break
        
#optimization    
def plot_count_neg_emojis2(chunks):
    for chunk in chunks:
            df = chunk[['description','tags','title']].count(negative_emojis)
            print(df)
            break                   

In [ ]:
#Suppose df_videos are only videos from 2019 ! 

df_videos_count_neg_emojis_desc = df_videos[df_videos.apply(lambda x: count_neg_emojis(x['description']), axis=1) > 0]
df_videos_count_neg_emojis_title = df_videos[df_videos.apply(lambda x: count_neg_emojis(x['title']), axis=1) > 0]

In [21]:
f = open("data/yt_metadata_en.jsonl")
videos = ijson.items(f, '', multiple_values=True)
list_count_neg_emojis = []

for video in tqdm.tqdm(videos):
    count_neg_emojis_desc = count_neg_emojis(video['description'])
    count_neg_emojis_title = count_neg_emojis(video['title'])
    list_count_neg_emojis.append({
        'count_neg_emojis_desc': count_neg_emojis_desc,
        'count_neg_emojis_title': count_neg_emojis_title
    })
f.close()    
print("finished") #40 min

72924794it [39:21, 30887.20it/s]


finished


NameError: name 'list_new_data' is not defined

In [22]:
filter_neg_emojis_df = pd.DataFrame.from_dict(list_count_neg_emojis)
filter_neg_emojis_df.to_csv('filtered_merges_neg_emoji.csv') #takes time as well !

KeyboardInterrupt: 

In [26]:
chunks = pd.read_json("data/yt_metadata_en.jsonl", lines=True, chunksize=1000000)

filtered_chunks = []
for chunk in tqdm.tqdm(chunks):
    df_filtered = chunk[chunk.apply(lambda x: count_neg_emojis_df(x['description'],x['title']), axis=1) > 0]
    filtered_chunks.append(df_filtered)
df_merged = pd.concat(filtered_chunks)    
df_merged.to_csv('filtered_merged_neg_emoji.csv')

73it [1:08:55, 56.65s/it] 


In [ ]:
filter_neg_emojis_df.head()

In [ ]:
f = open(data_path + 'yt_metadata_en.jsonl')
output = open('generated/videos_few_columnstmp.csv', 'w')

videos = ijson.items(f, '', multiple_values=True)

columns_to_drop = [
    'description', 
    'tags', 
    'title',
    'display_id',
    'duration',
    'crawl_date'
]
    
columns_to_keep = [
    'categories', 
    'channel_id', 
    'dislike_count', 
    'like_count',
    'upload_date',
    'view_count'

]

columns = [f'"{str(col)}"' for col in columns_to_keep]
output.write(','.join(columns))
output.write('\n')

for video in tqdm(videos):
    
    data_to_write = [str(video[field]) for field in columns_to_keep]
    output.write(','.join(data_to_write))
    output.write('\n')
    
    
print('Finished') # 11min05s
output.close()
f.close()


types_col = {    
    'categories': 'string', 
    'channel_id': 'string', 
    'dislike_count': 'Int32',
    'like_count': 'Int64',
    'upload_date': 'string', 
    'view_count': 'Int64'
}
v = pd.read_csv(
    'generated/videos_few_columns.csv', 
    sep=',', 
    header=0, 
    dtype=types_col, 
    na_values=['None'], 
    engine='c'
)

v.to_parquet('generated/videos_few_columns.parquet', compression=None)

In [ ]:
all_videos = pd.read_parquet('generated/videos_few_columns.parquet', engine='fastparquet')

In [ ]:
joined = all_videos.join(filter_neg_emojis_df)

In [ ]:
joined.to_parquet('generated/videos_CountNegWordsTitle.parquet', compression=None)

## Plots

In [ ]:
joined.head()